<a href="https://www.kaggle.com/code/jyotsnacd/coffee-quality-brew-data-cleaning-visualization?scriptVersionId=133718173" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 50)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from scipy import stats
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Loading Data and checking samples**

In [ ]:
filepath = "/kaggle/input/coffee-quality-data-cqi/df_arabica_clean.csv"
df = pd.read_csv(filepath)
df.head(3)

In [ ]:
# making ID as DF index and removing Unnamed columns
df.set_index("ID",inplace = True)
df.drop("Unnamed: 0",axis=1,inplace=True)


In [ ]:
#Let's all see how categorical variales are loooking
df.describe(include="object")

###  Observations
1. The columns "Company" and "Owner" provide redundant information, with the additional inclusion of Chinese names in the "Owner" column. Therefore,   it is safe to remove the "Owner" column from the dataset. <br>
2. Bag weight seems to be numeric."kg" suffix need to be removed and data type should be changed it float  <br>
3. ["Producer","In-Country Partner","Certification Body","Farm Name","Mill","Region"] have some chinese character. They need cleaning.<br>
4. "Status" column has no information. So we can drop it.<br>
5. "Grading Date","Expiration","Harvest Year" need to be converted in Date Format.<br>
6. "Certification Address" and "Certification Contact" can be deleted as we have region column and contact person seems irrelevant here.<br>
7. "Harvest Year" sometimes have values in the form of ranges. We can keep the first value in order to have the values of columns as a single year.<br>
8. As we can see that more than half of ICO Number is Null, so we can straightaway drop the column.


In [ ]:
#removing columns "Owner", "Certification Address", "Certification Contact", "Status"
try:
    df.drop(["Owner", "Certification Address", "Certification Contact", "Status","Lot Number","ICO Number"],axis=1, inplace=True)
except KeyError:
    pass

# removing kg from Bag weight column and converting to int dtype
try:
    df["Bag Weight"] = df["Bag Weight"].apply(lambda x: x.split(" ")[0]).astype("int64")
except AttributeError:
    pass

# removing chinese character from "In-company Partner" and "Producer"
import re 
#to extract only english characters
def extract_eng_char(s):
    s = str(s)
    s = re.sub(r'[^a-zA-Z0-9\s]', '', s).strip()
    return s

columns_with_noneng = ["Producer","In-Country Partner","Certification Body","Farm Name","Mill","Region"]
for col in columns_with_noneng:
    df[col] = df[col].apply(extract_eng_char)
    #Set unknown as producer for empty strings
    df[col] = df[col].replace("","unknown")


#let's keep a copy of df so far
df_1 = df.copy()

#"Expiration" needs to be converted in Date Format.
date_cols = ["Grading Date","Expiration"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col])
    
#Keeping first value of harvest value
df["Harvest Year"] = df["Harvest Year"].apply(lambda x: x.strip().split("/")[0].strip() if "/" in x.strip() else x.strip())
df["Harvest Year"] = pd.to_datetime(df["Harvest Year"],format="%Y")




* Let's see all the numerical and categorical columns

In [ ]:
#drop columns with unique value = 1
cols_to_drop = df.columns[df.nunique()==1]
df.drop(cols_to_drop,axis=1, inplace = True)

In [ ]:
df.describe(include="object")

**Observations in Categorical Columns**
* Among all, we can observe that ["Farm Name", "Mill", "Region", "Producer"] have majority values as unknown. However, let's not discard it right away.
* Except for Color and Processing Methods, there seems to be too many categories of each column. We can later check if we can merge under-represented column values
*Altitude appears to be a numerical column.and there are also some range altitudes. let's take the average and convert the dtype to int right away

In [ ]:
#Calculating Average Altitude where range is given
def get_avg_alt(x):
    x = str(x)
    x = re.findall(r'\d+|\D+', x)
    if len(x) ==3:
        return (int(x[0])+int(x[2]))//2
    elif len(x) == 1 and x[0].isdigit():
        return int(x[0])
    else:
        return -1
df["Altitude"] = df["Altitude"].apply(get_avg_alt)

#As we can see "Processing Method", "Variety" have a very few missing values. 
df["Processing Method"] = df["Processing Method"].fillna("Unknown")
df["Variety"] = df["Variety"].fillna("Unknown")

In [ ]:
numerical_cols = df.select_dtypes(include=['float', 'int']).columns
non_num_cols = df.select_dtypes(include=['object']).columns
date_cols = df.select_dtypes(include=['datetime64']).columns

print(numerical_cols,end="\n\n")
print(non_num_cols,end="\n\n")
print(date_cols,end="\n")

# Numerical Variables

In [ ]:
#Visualizing Distributions for numerical coulmns

fig,axs = plt.subplots(4,4,figsize=(12,12))
plt.subplots_adjust(wspace=0.8,hspace=0.8)

for i,col in enumerate(numerical_cols):
    sns.histplot(df[col],color="skyblue", ax = axs[int(i/4),i%4]).set_ylabel("")
plt.xticks(rotation=45)
plt.show()

**Observations:**
* "Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance" and "Total Cups" appear to be normally distributed. We can check for normality test
* "Moisture Percentage" and "Altitude" seem to be normally distributed too except for a few outliers.
* "Category Two defects" are exponential distributed
* "Uniformity" and "Catgory One Defects" have majority as a single value except for a few outliers. So we can discard these columns considering they don't carry any useful information.
* "Bag Weight" seems to have outliers. hence we need to remove them.

In [ ]:
#First Let's perform the normality Test ("Shapiro-Wilk Test") to confirm the normality of columns:
#["Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance" and "Total Cups"]

cols_to_test = ["Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance" and "Total Cup Points", "Moisture Percentage", "Altitude"]

for col in cols_to_test:
    statistic, p_value = stats.shapiro(df[col])
    # Check the p-value
    alpha = 0.05  # Set the significance level
    if p_value > alpha:
        print(f"{col} values are normally distributed (failed to reject H0)",end="\n")
    else:
        print(f"{col} values are not normally distributed (may reject H0)",end="\n")


Results: **None of the above columns follow Normal Distribution except "Total Cup Points"**

In [ ]:
# Let's now handle the outliers in the aforementioned columns : "Bag Weight", "Moisture Percentage" and "Altitude"
#Let's replace the outliers with mean values. We can assume lower and upper bound thresholds of outliers as 5th percentile and 95th percentile values

cols_with_outliers = ["Bag Weight", "Moisture Percentage", "Altitude","Aroma","Quakers", "Balance", "Number of Bags"]
for col in cols_with_outliers:
    lower_thresh = np.percentile(df[col],5)
    upper_thresh = np.percentile(df[col],95)
    mean_val = np.mean(df[col])
    df.loc[(df[col] <= lower_thresh) | (df[col] >= upper_thresh), col] = mean_val

#Also, dropping columns "Uniformity" and "Catgory One Defects" since they don't carry any useful info
# df.drop("Uniformity",axis=1,inplace=True)
cols_to_drop = ["Uniformity", "Category One Defects"]
#df.drop("Category One Defects",axis=1,inplace=True)


numerical_cols = numerical_cols[(numerical_cols!="Uniformity") & (numerical_cols!="Category One Defects")]
#numerical_cols.remove(["Category One Defects"],axis=1, inplace=True)
    

In [ ]:
#Visualizing Distributions after handling outliers

fig,axs = plt.subplots(4,4,figsize=(12,12))
plt.subplots_adjust(wspace=0.8,hspace=0.8)

for i,col in enumerate(numerical_cols):
    sns.histplot(df[col],color="skyblue", ax = axs[int(i/4),i%4]).set_ylabel("")
fig.delaxes(axs[3, 2])
fig.delaxes(axs[3, 3])
plt.xticks(rotation=45)
plt.show()

### Pre-processing Catgorical Variables

In [ ]:
#Let's check the summary statistics of Categorical variables
df.describe(include="object")

In [ ]:
#Categorical Variables Distribution
fig,axs = plt.subplots(4,3,figsize=(12,12))
plt.subplots_adjust(wspace=0.8,hspace=0.8)

for i,col in enumerate(non_num_cols):
    sns.countplot(x=col,data=df,color="skyblue", ax = axs[int(i/3),i%3]).set_ylabel("")
    
    
# Hide x-ticks for each subplot
for ax in axs.flat:
    ax.set_xticks([])
fig.delaxes(axs[3, 2])
plt.xticks([])
plt.show()




**Observations:**
* Each of the Categorical Variable has one or two dominating values  while the other values represent a smaller proportion of data.
* "Farmname", "Mill", "Region" and "Producer" have missing values/chinese characters which we had replaced with "Unknown" earlier while cleaning the data
* We can combine the minority elements into a single category value in the columns where data is highly imbalanced, or we can also consider setting a lower threshold count to combine categories

In [ ]:
#Combining categories for each column where represent less than 10% of datapoints
threshold = 5
for col in non_num_cols:
    col_freq = df[col].value_counts()
    to_be_changed = col_freq[col_freq<5].index
    df[col] = df[col].replace(to_be_changed,"Others")
    
    

In [ ]:
#Categorical Variables Distribution
fig,axs = plt.subplots(4,3,figsize=(16,16))
plt.subplots_adjust(wspace=0.8,hspace=0.8)

for i,col in enumerate(non_num_cols):
    cat_sorted = df[col].value_counts().index
    sns.countplot(x=col,data=df,color="skyblue", ax = axs[int(i/3),i%3], order = cat_sorted ).set_ylabel("")
    
    
for ax in axs.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45,fontsize=5)
    
fig.delaxes(axs[3, 2])
plt.show()

In [ ]:
#Visualizing the correlation between numerical features
corr_matrix = df[numerical_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)

### Observations:
* It is evident that there exists a strong correlation among the properties that define the quality of coffee.
* By examining the correlation between "Total Cup Points" and various characteristics of coffee, we can gain insights into the relative importance assigned to each quality test when determining the points for the overall quality evaluation.

# Conclusion: <br>
**1. The dataset offers a comprehensive overview of the necessary data for analyzing coffee quality, encompassing all relevant coffee properties and   their correlations, thereby providing a holistic perspective.** <br>
**2. The categorical columns in the dataset exhibit a significant imbalance, with a small number of values dominating the majority of data points.**<br>
**3. After performing the Shapiro-Wilk test, it was found that only the "Total Cup Points" column follows a normal distribution, while the other numerical columns deviate from normality even though they look approximately normal.**<br>
**4. It is necessary to address outliers and perform data cleaning for the numerical columns.**<br>
**5. Certain columns, such as "Uniformity" and "Category One Defects," contain only a single value with some outliers, making them uninformative. As a result, these columns can be safely discarded from the analysis.**<br>



*PS - If you appreciate the work in my notebook, I kindly request you to consider upvoting it. Your support would be greatly appreciated. Thank you!*